# UCI MTB DH Data Retrieval

## Setup
#### Import Libraries

If you do not have these libraries available, you should install them using `pip`

```
pip install requests
pip install bs4
pip install pandas
```

In [365]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import os

In [366]:
def calculate_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

Widen display area to prevent column wrapping, and always show all columns for debug 

In [367]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

## Config

Which race data are we collecting?

1. Losinj
1. Fort William
1. Leogang
1. Val di Sole
1. Vallnord
1. Mont-Sainte-Anne
1. La Bresse

In [415]:
race = 2
gender = 'f'
event = 'dh'

#### Data Sources

The UCI Live Timing API contains a lot of data points, but not all the ones we want (speed being the main one missing), and not even all the ones they include on their own PDF which is frustrating.

Similarly, Roots & Rain also has a lot of the data points, but again not all of them; most notably it's missing timing splits 4 and 5.

Therefore we need to pull from both sources and combine the sets.

We specify the URLs for both sources from which we will extract our data. The UCI API URL can be found by loading the Live Timing page then using your browser's inspector on the Network tab (in Chrome at least) to see the data feed. As the UCI seems to be using a Single Page Application (SPA) here, it's not straight forward to extract this link automagically.

**Note:** The Race list is now maintained as an external python config file `config.py` imported below

In [416]:
from config import races

racename = races[race]['name']
urlUci = races[race]['urls']['uci'] + str(( 3 if 'm' == gender else 6 )) + '/'
urlUciQ = races[race]['urls']['uci'] + str(( 2 if 'm' == gender else 5 )) + '/'
urlRoots = races[race]['urls']['rnr'] + gender + '/'

File handling setup

In [417]:
directory = event + str(race) + '_' + racename
if not os.path.exists(directory):
    os.makedirs(directory)

file_prefix = event + str(race) + '_' + racename + '_' + gender
file_prefix = os.path.join( directory, file_prefix )

# UCI API
### Load Data

These two lines make the actual request to the server, and then converts the JSON string response in to a usable list format (deserialization)

In [418]:
r = requests.get( urlUci ).json()
q = requests.get( urlUciQ ).json()

The API returns with three main sections:

1. `Last Finisher`
 - Racers in order of start time
2. `Results`
 - Racers in finishing rank order
3. `Riders`
 - Personal details on all racers
 
Each contains many data points. To see all the contained data, you can un-comment and execute any of the lines in the next section to explore more.

In [419]:
# display( q )
# display( d['Results'][7] )
# display( d['Riders']['1001'] )
# display( d['Results'][61] )

### Extract Data

Here we iterate over the `Results` sub-set of data to extract the information we care about: basically some identifying info, and their splits.

There is a loop within a loop here as the first iterates over the two result sets qualifying and race, and within that we extract the necessary stats.

If you looked at detail of the returned data set in the last step you might have noticed the rider's name is not stored next to their result, riders are only identified by a reference number. To facilitate our analysis later on it is useful to import each rider's name at this stage by cross-referencing the `Riders` sub-set.

We start with an empty list `lst` and in each loop iteration add an entry (actually a dict) to that list for each rider.

In [420]:
dat = {}
for i, d in enumerate( [ r, q ] ):
    lastStart = d['Riders'][list(d['Riders'].keys())[-1]]['StartOrder']
    pfx = 'q_' if 1 == i else ''

    splits = len(d['Results'][0]['Times'] )
    lst = []
    for idx, row in enumerate( d['Results'] ):
        fin = "Finished" == row['Status']
        res = {
            'name': d['Riders'][str(row['RaceNr'])]['PrintName'],
            'id': row['RaceNr'],
            'uci': d['Riders'][str(row['RaceNr'])]['UciRiderId'],
            'bib': d['Riders'][str(row['RaceNr'])]['RaceNr'],
            pfx + 'status': row['Status'],
            pfx + 'rank': row['Position'] if fin else idx+1,
            pfx + 'speed': np.nan,
            pfx + 'start': d['Riders'][str(row['RaceNr'])]['StartOrder'],
            pfx + 'start_rev': lastStart - d['Riders'][str(row['RaceNr'])]['StartOrder'] +1
        }

        # Add all splits to result set
        for split in range( 0, splits ):
            head = pfx + 'split'
            res[head + str(split+1)] = row['Times'][split]['RaceTime']/1000 if fin else np.nan

        # Append result set to list
        lst.append(res)

    dat[i] = lst

Here we load the completed lists in to Pandas dataframes to facilitate working with the data moving forward

In [421]:
df = pd.DataFrame( dat[0] )
dq = pd.DataFrame( dat[1] )

# Points

Neither data set contains points awarded so we use a reference file and merge

Merge type here must be `outer` so people that finished outside the top 60 men, top 15 women, or DNF, don't get trimmed from the dataset

In [422]:
df_qp = pd.read_csv( event + '_points_qual_' + gender + '.csv', index_col=0 )
df_qp = df_qp.reset_index(drop=False)
dq = dq.merge( df_qp, left_index=True, right_index=True, how="outer")

df_rp = pd.read_csv( event + '_points_race_' + gender + '.csv', index_col=0 )
df_rp = df_rp.reset_index(drop=False)
df = df.merge( df_rp, left_index=True, right_index=True, how="outer")

In [423]:
display( df.head(), dq.head() )

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,r_points
0,3,2003,SEAGRAVE Tahnee,1,NaN,64.814,211.338,240.134,281.291,310.960,14,3,Finished,10007414016,200.0
1,1,2001,NICOLE Myriam,2,NaN,65.404,216.908,247.122,288.983,319.211,15,2,Finished,10004535237,160.0
2,2,2002,ATHERTON Rachel,3,NaN,65.120,218.276,247.811,289.281,320.016,16,1,Finished,10003434487,140.0
3,4,2004,CABIROU Marine,4,NaN,68.834,219.147,249.423,291.659,321.782,11,6,Finished,10009563069,125.0
4,7,2007,HANNAH Tracey,5,NaN,69.640,221.078,251.784,292.907,322.744,13,4,Finished,10003732258,110.0


,bib,id,name,q_rank,q_speed,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points
0,2,2002,ATHERTON Rachel,1,NaN,64.790,208.081,237.522,277.644,307.825,2,22,Finished,10003434487,50.0
1,1,2001,NICOLE Myriam,2,NaN,64.731,207.479,236.619,278.295,309.765,1,23,Finished,10004535237,40.0
2,3,2003,SEAGRAVE Tahnee,3,NaN,65.784,215.552,244.961,286.927,316.908,3,21,Finished,10007414016,30.0
3,7,2007,HANNAH Tracey,4,NaN,68.024,215.659,245.135,287.062,316.984,7,17,Finished,10003732258,25.0
4,5,2005,RAVANEL Cecile,5,NaN,69.712,220.343,249.447,291.609,322.395,5,19,Finished,10002816317,20.0


# Merge and Expand

This code merges the qualifying and race data in to a single data frame, merging only the columns that are unique between them to avoid duplicates. This allows us to do more in depth analysis later on.

As we merged race in to quali, we re-sort the resulting dataset by race rank

In [424]:
dfq = dq.merge( df[['id'] + list(df.columns.difference( dq.columns ))], left_on='id', right_on='id', how='outer' )
dfq = dfq.sort_values( 'rank', ascending=True )
dfq = dfq.reset_index( drop=True )
dfq['points'] = dfq['r_points'].fillna(0) + dfq['q_points'].fillna(0)

# Time difference between race and quali
dfq['qr_diff'] = dfq['split5'] - dfq['q_split5']

#### Expand Dataset

Calculate and add all the extra columns we need for split and sector differences and their rankings

In [425]:
for pfx in [ 'q_', '' ]:
    for i in range( 1, splits+1 ):
        split = pfx + 'split' + str(i)
        sector = split + '_sector'
        dfq[split + '_rank'] = dfq[split].rank(method='dense')
        dfq[split + '_vs_best'] = (dfq[split] - dfq[split].min())
        dfq[split + '_vs_winner'] = (dfq[split] - dfq[split][0])

        if i > 1:
            dfq[split + '_sector'] = dfq[split] - dfq[pfx + 'split' + str(i-1)]
            dfq[split + '_sector_rank'] = dfq[sector].rank(method='dense')
            dfq[split + '_sector_vs_best'] = (dfq[sector] - dfq[sector].min())
            dfq[split + '_sector_vs_winner'] = (dfq[sector] - dfq[sector][0])

We can take a peek at our data at this point to make sure it looks how we expect.

At this point the `speed` column is NaN (Not a Number) for all racers. This will be filled in below.

In [426]:
display( dfq.head(10) )

,bib,id,name,q_rank,q_speed,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points,r_points,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,points,qr_diff,q_split1_rank,q_split1_vs_best,q_split1_vs_winner,q_split2_rank,q_split2_vs_best,q_split2_vs_winner,q_split2_sector,q_split2_sector_rank,q_split2_sector_vs_best,q_split2_sector_vs_winner,q_split3_rank,q_split3_vs_best,q_split3_vs_winner,q_split3_sector,q_split3_sector_rank,q_split3_sector_vs_best,q_split3_sector_vs_winner,q_split4_rank,q_split4_vs_best,q_split4_vs_winner,q_split4_sector,q_split4_sector_rank,q_split4_sector_vs_best,q_split4_sector_vs_winner,q_split5_rank,q_split5_vs_best,q_split5_vs_winner,q_split5_sector,q_split5_sector_rank,q_split5_sector_vs_best,q_split5_sector_vs_winner,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner
0,3,2003,SEAGRAVE Tahnee,3,NaN,65.784,215.552,244.961,286.927,316.908,3,21,Finished,10007414016,30.0,200.0,1.0,NaN,64.814,211.338,240.134,281.291,310.960,14.0,3.0,Finished,230.0,-5.948,3.0,1.053,0.000,3.0,8.073,0.000,149.768,4.0,7.020,0.000,3.0,8.342,0.000,29.409,3.0,0.305,0.000,3.0,9.283,0.000,41.966,4.0,1.844,0.000,3.0,9.083,0.000,29.981,2.0,0.059,0.000,1.0,0.000,0.000,1.0,0.000,0.000,146.524,1.0,0.000,0.000,1.0,0.000,0.000,28.796,1.0,0.000,0.000,1.0,0.000,0.000,41.157,2.0,0.034,0.000,1.0,0.000,0.000,29.669,1.0,0.000,0.000
1,1,2001,NICOLE Myriam,2,NaN,64.731,207.479,236.619,278.295,309.765,1,23,Finished,10004535237,40.0,160.0,2.0,NaN,65.404,216.908,247.122,288.983,319.211,15.0,2.0,Finished,200.0,9.446,1.0,0.000,-1.053,1.0,0.000,-8.073,142.748,1.0,0.000,-7.020,1.0,0.000,-8.342,29.140,2.0,0.036,-0.269,2.0,0.651,-8.632,41.676,2.0,1.554,-0.290,2.0,1.940,-7.143,31.470,8.0,1.548,1.489,3.0,0.590,0.590,2.0,5.570,5.570,151.504,5.0,4.980,4.980,2.0,6.988,6.988,30.214,4.0,1.418,1.418,2.0,7.692,7.692,41.861,4.0,0.738,0.704,2.0,8.251,8.251,30.228,7.0,0.559,0.559
2,2,2002,ATHERTON Rachel,1,NaN,64.790,208.081,237.522,277.644,307.825,2,22,Finished,10003434487,50.0,140.0,3.0,NaN,65.120,218.276,247.811,289.281,320.016,16.0,1.0,Finished,190.0,12.191,2.0,0.059,-0.994,2.0,0.602,-7.471,143.291,2.0,0.543,-6.477,2.0,0.903,-7.439,29.441,4.0,0.337,0.032,1.0,0.000,-9.283,40.122,1.0,0.000,-1.844,1.0,0.000,-9.083,30.181,3.0,0.259,0.200,2.0,0.306,0.306,3.0,6.938,6.938,153.156,8.0,6.632,6.632,3.0,7.677,7.677,29.535,2.0,0.739,0.739,3.0,7.990,7.990,41.470,3.0,0.347,0.313,3.0,9.056,9.056,30.735,11.0,1.066,1.066
3,4,2004,CABIROU Marine,6,NaN,69.122,219.452,248.975,291.874,322.750,4,20,Finished,10009563069,16.0,125.0,4.0,NaN,68.834,219.147,249.423,291.659,321.782,11.0,6.0,Finished,141.0,-0.968,6.0,4.391,3.338,5.0,11.973,3.900,150.330,5.0,7.582,0.562,5.0,12.356,4.014,29.523,6.0,0.419,0.114,6.0,14.230,4.947,42.899,8.0,2.777,0.933,6.0,14.925,5.842,30.876,6.0,0.954,0.895,5.0,4.020,4.020,4.0,7.809,7.809,150.313,2.0,3.789,3.789,4.0,9.289,9.289,30.276,5.0,1.480,1.480,4.0,10.368,10.368,42.236,6.0,1.113,1.079,4.0,10.822,10.822,30.123,5.0,0.454,0.454
4,7,2007,HANNAH Tracey,4,NaN,68.024,215.659,245.135,287.062,316.984,7,17,Finished,10003732258,25.0,110.0,5.0,NaN,69.640,221.078,251.784,292.907,322.744,13.0,4.0,Finished,135.0,5.760,4.0,3.293,2.240,4.0,8.180,0.107,147.635,3.0,4.887,-2.133,4.0,8.516,0.174,29.476,5.0,0.372,0.067,4.0,9.418,0.135,41.927,3.0,1.805,-0.039,4.0,9.159,0.076,29.922,1.0,0.000,-0.059,8.0,4.826,4.826,6.0,9.740,9.740,151.438,4.0,4.914,4.914,6.0,11.650,11.650,30.706,7.0,1.910,1.910,5.0,11.616,11.616,41.123,1.0,0.000,-0.034,5.0,11.784,11.784

#### Rider Data

Saving the personal information about each racer is much easier as we can just export the entire `Riders` dataset. However, the rows and columns are the wrong way round so the `.T` command *transposes* the information, meaning it basically flips the axes.

In [427]:
df2 = pd.DataFrame( d['Riders'] )
df2 = df2.T
df2['Age'] = [ calculate_age( dt.datetime.strptime( dob[:10], "%Y-%m-%d" ) ) for dob in df2['BirthDate'] ]

Here we can glimpse the first few rows of our `DataFrame` and can check the data looks as we expect

In [428]:
display( df2.head() )

,BirthDate,CategoryCode,FamilyName,GivenName,Id,Nation,Outfit,PrintName,RaceId,RaceNr,ScoreboardName,StartOrder,StartTime,UciCode,UciRank,UciRiderId,UciTeamCode,UciTeamId,UciTeamName,WorldCupRank,Age
2001,1990-02-08T00:00:00,WE,NICOLE,Myriam,1196990205528238,FRA,WCL,NICOLE Myriam,0,1,NICOLE M,1,48600000,FRA19900208,1,10004535237,CVN,1590,COMMENCAL / VALLNORD,1,28
2002,1987-12-06T00:00:00,WE,ATHERTON,Rachel,1196990205528239,GBR,NCh,ATHERTON Rachel,0,2,ATHERTON R,2,48630000,GBR19871206,6,10003434487,TDH,1598,TREK FACTORY RACING DH,2,30
2003,1995-06-15T00:00:00,WE,SEAGRAVE,Tahnee,1196990205528240,GBR,NaN,SEAGRAVE Tahnee,0,3,SEAGRAVE T,3,48660000,GBR19950615,2,10007414016,FMD,1863,TRANSITION BIKES / MUC-OFF FACTORY RACING,3,22
2004,1997-03-12T00:00:00,WE,CABIROU,Marine,1196990205528241,FRA,NaN,CABIROU Marine,0,4,CABIROU M,4,48690000,FRA19970312,4,10009563069,VVR,2096,VOULVOUL RACING,4,21
2005,1981-01-06T00:00:00,WE,RAVANEL,Cecile,1196990205528242,FRA,NaN,RAVANEL Cecile,0,5,RAVANEL C,5,48720000,FRA19810106,38,10002816317,,0,,5,37


# Roots and Rain
### Load Data

Similar to the UCI api, we make a request to the server with the previously declared `urlRoots` variable. This time however we simply load the content of the response as text which is actually the HTML code of the web page. We do not do have a nice JSON API to read which means we will not deserialize.

Next we invoke a utility called `BeautifulSoup` to help us extract the data from this messy HTML code

In [429]:
c = requests.post( urlRoots ).content
soup = BeautifulSoup( c, "html.parser" )

### Extract Data

If you look at the Roots and Rain page you'll see it listed in a tabular format. What we do here is find all the rows of that table so we can extract the information we need.

Specifically we are looking for instances of `tr` (table row), with a class that *begins with* `c-` as this is a common denomenator I discovered when looking through the code with the browser inspector

In [430]:
rows = soup.find_all( "tr", class_=lambda x: x and 'c-' in x )

Similar to the UCI data set, here we will iterate over each row in our data set--basically each table row from the web page--and extract the bits we need.

Racer speed is the metric we're interested in, but in order to match that to our existing data set we need a corresponding identifier so we also extract the racer licence number as that exists in both sets and we can match them together: it is the *intersect* between both sets of data.

To summarise:
1. Extract licence number and corresponding speed
2. Import speed to existing DataFrame matching racers by licence

The `if` condition in the middle will exit this block of code once we hit the end of the Elite finishers, seeing as that's all we have in our existing data set so can't match anyone else

In [431]:
for row in rows:
    cells = row.find_all( "td" )
    qspd = cells[7].text[:5]
    spd = cells[12].text[:5]
    qspeed = float( qspd if 0 < len(qspd) else 0 )
    speed = float( spd if 0 < len(spd) else 0 )
    licence = cells[4].text
    bib = int( cells[1].text )
    pos = cells[0].text[8:]
    if "" == pos: break

    # Match rider by UCI licence if present, otherwise fallback to bib
    if len(df2.loc[df2['UciRiderId'] == licence].index.values ):
        rid = int(df2.loc[df2['UciRiderId'] == licence].index.values[0])
    else:
        rid = int( df2.loc[df2['RaceNr'] == bib].index.values[0] )

    # Add speed, and other associated metrics
    dfq.loc[dfq['id'] == rid, 'speed'] = speed
    dfq.loc[dfq['id'] == rid, 'q_speed'] = qspeed
    dfq.loc[dfq['id'] == rid, 'speed_ms'] = float(speed)*(1000/60/60)
    dfq.loc[dfq['id'] == rid, 'speed_ms_vs_best'] = dfq['speed_ms'].max() - dfq.speed_ms
    dfq['speed_rank'] = dfq.speed.rank(method='dense', ascending=False)
    dfq['q_speed_rank'] = dfq.q_speed.rank(method='dense', ascending=False)

As before, we can take another look at how our data is looking, with the `speed` column now containing data 

In [432]:
display( dfq.head() )

,bib,id,name,q_rank,q_speed,q_split1,q_split2,q_split3,q_split4,q_split5,q_start,q_start_rev,q_status,uci,q_points,r_points,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,points,qr_diff,q_split1_rank,q_split1_vs_best,q_split1_vs_winner,q_split2_rank,q_split2_vs_best,q_split2_vs_winner,q_split2_sector,q_split2_sector_rank,q_split2_sector_vs_best,q_split2_sector_vs_winner,q_split3_rank,q_split3_vs_best,q_split3_vs_winner,q_split3_sector,q_split3_sector_rank,q_split3_sector_vs_best,q_split3_sector_vs_winner,q_split4_rank,q_split4_vs_best,q_split4_vs_winner,q_split4_sector,q_split4_sector_rank,q_split4_sector_vs_best,q_split4_sector_vs_winner,q_split5_rank,q_split5_vs_best,q_split5_vs_winner,q_split5_sector,q_split5_sector_rank,q_split5_sector_vs_best,q_split5_sector_vs_winner,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner,speed_ms,speed_ms_vs_best,speed_rank,q_speed_rank
0,3,2003,SEAGRAVE Tahnee,3,50.77,65.784,215.552,244.961,286.927,316.908,3,21,Finished,10007414016,30.0,200.0,1.0,52.20,64.814,211.338,240.134,281.291,310.960,14.0,3.0,Finished,230.0,-5.948,3.0,1.053,0.000,3.0,8.073,0.000,149.768,4.0,7.020,0.000,3.0,8.342,0.000,29.409,3.0,0.305,0.000,3.0,9.283,0.000,41.966,4.0,1.844,0.000,3.0,9.083,0.000,29.981,2.0,0.059,0.000,1.0,0.000,0.000,1.0,0.000,0.000,146.524,1.0,0.000,0.000,1.0,0.000,0.000,28.796,1.0,0.000,0.000,1.0,0.000,0.000,41.157,2.0,0.034,0.000,1.0,0.000,0.000,29.669,1.0,0.000,0.000,14.500000,0.000000,3.0,3.0
1,1,2001,NICOLE Myriam,2,50.45,64.731,207.479,236.619,278.295,309.765,1,23,Finished,10004535237,40.0,160.0,2.0,50.72,65.404,216.908,247.122,288.983,319.211,15.0,2.0,Finished,200.0,9.446,1.0,0.000,-1.053,1.0,0.000,-8.073,142.748,1.0,0.000,-7.020,1.0,0.000,-8.342,29.140,2.0,0.036,-0.269,2.0,0.651,-8.632,41.676,2.0,1.554,-0.290,2.0,1.940,-7.143,31.470,8.0,1.548,1.489,3.0,0.590,0.590,2.0,5.570,5.570,151.504,5.0,4.980,4.980,2.0,6.988,6.988,30.214,4.0,1.418,1.418,2.0,7.692,7.692,41.861,4.0,0.738,0.704,2.0,8.251,8.251,30.228,7.0,0.559,0.559,14.088889,0.411111,8.0,5.0
2,2,2002,ATHERTON Rachel,1,50.61,64.790,208.081,237.522,277.644,307.825,2,22,Finished,10003434487,50.0,140.0,3.0,49.81,65.120,218.276,247.811,289.281,320.016,16.0,1.0,Finished,190.0,12.191,2.0,0.059,-0.994,2.0,0.602,-7.471,143.291,2.0,0.543,-6.477,2.0,0.903,-7.439,29.441,4.0,0.337,0.032,1.0,0.000,-9.283,40.122,1.0,0.000,-1.844,1.0,0.000,-9.083,30.181,3.0,0.259,0.200,2.0,0.306,0.306,3.0,6.938,6.938,153.156,8.0,6.632,6.632,3.0,7.677,7.677,29.535,2.0,0.739,0.739,3.0,7.990,7.990,41.470,3.0,0.347,0.313,3.0,9.056,9.056,30.735,11.0,1.066,1.066,13.836111,0.663889,11.0,4.0
3,4,2004,CABIROU Marine,6,49.30,69.122,219.452,248.975,291.874,322.750,4,20,Finished,10009563069,16.0,125.0,4.0,50.55,68.834,219.147,249.423,291.659,321.782,11.0,6.0,Finished,141.0,-0.968,6.0,4.391,3.338,5.0,11.973,3.900,150.330,5.0,7.582,0.562,5.0,12.356,4.014,29.523,6.0,0.419,0.114,6.0,14.230,4.947,42.899,8.0,2.777,0.933,6.0,14.925,5.842,30.876,6.0,0.954,0.895,5.0,4.020,4.020,4.0,7.809,7.809,150.313,2.0,3.789,3.789,4.0,9.289,9.289,30.276,5.0,1.480,1.480,4.0,10.368,10.368,42.236,6.0,1.113,1.079,4.0,10.822,10.822,30.123,5.0,0.454,0.454,14.041667,0.458333,9.0,8.0
4,7,2007,HANNAH Tracey,4,51.20,68.024,215.659,245.135,287.062,316.984,7,17,Finished,10003732258,25.0,110.0,5.0,53.08,69.640,221.078,251.784,292.907,322.744,13.0,4.0,Finished,135.0,5.760,4.0,3.293,2.240,4.0,8.180,0.107,147.635,3.0,4.887,-2.133,4.0,8.516,0.174,29.476,5.0,0.372,0.067,4.0,9.418,0.135,41.927,3.0,1.805,-0.039,4.0,9.159,0.076,2

# Data Export

All that's left is to save our data to CSV files so we can quickly import it again for analysis and visualization without making constant requests to the online servers. This not only reduces load on the services providing the data, but also allows us to work on our analysis "offline", moreover giving us a local copy in case the results are ever taken down. It's also much quicker to load data this way than constantly hitting online servers.

In [433]:
dfq.id = dfq.id.astype(str)
dfm = dfq.merge( df2, left_on='id', right_index=True, how='inner' )

In [434]:
df.to_csv( file_prefix + '.results.csv' )
dq.to_csv( file_prefix + '.quali.csv' )
df2.to_csv( file_prefix + '.racers.csv' )
dfm.to_csv( file_prefix + '.merged.csv' )

--- 

## Credits

### Author: Dominic Wrapson


> **@domwrap**
<br>
<img src="https://png.icons8.com/material/24/000000/github-2.png">
<img src="https://png.icons8.com/material/24/000000/stackoverflow.png">
<img src="https://png.icons8.com/material/24/000000/linkedin.png">
<img src="https://png.icons8.com/material/24/000000/windows8.png">
<img src="https://png.icons8.com/ios-glyphs/24/000000/instagram-new.png">
<img src="https://png.icons8.com/material/24/000000/twitter.png">
<a href="https://medium.com/@domwrap"><img src="https://png.icons8.com/material/24/000000/medium-logo.png"></a>
>
> <img src="https://png.icons8.com/material/24/000000/home.png"> http://domwrap.me
>
><img src="https://png.icons8.com/material/24/000000/cycling-mountain-bike.png"> [Hwulex](https://www.pinkbike.com/u/Hwulex/)


---

#### Special Thanks

Mark Shilton for the inspiration
- http://lookatthestats.blogspot.ca
- https://plus.google.com/+MarkShilton
- https://dirtmountainbike.com/author/mrgeekstats


<a href="https://icons8.com">Icon pack by Icons8</a>